In [12]:
#Basic needed libraries
import pandas as pd

#Metrics and models
from sklearn.metrics import silhouette_score
from kmodes.kmodes import KModes
from kmodes.kprototypes import KPrototypes

In [16]:
data_list = [
    [1, 'M', 23, 'SP'],
    [2, 'F', 45, 'RJ'],
    [3, 'F', 31, 'MG'],
    [4, 'M', 29, 'SP'],
    [5, 'F', 35, 'RJ'],
    [6, 'M', 41, 'MG']
]

data = pd.DataFrame(data=data_list)

In [19]:
data_numeric = data.select_dtypes(include='number')
data_categorical = data.select_dtypes(include='object')

In [20]:
data_categorical

,1,3
0,M,SP
1,F,RJ
2,F,MG
3,M,SP
4,F,RJ
5,M,MG
